<a href="https://colab.research.google.com/github/retrouvailles00/cs6120/blob/main/RND_CS6120_RuiLiu_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import json,requests,argparse
import os,sys,argparse,time
import numpy as np
from collections import defaultdict


In [32]:
df = pd.read_csv('limited_papers.csv')
apikey=os.environ.get('SPECTER_API_KEY')

In [33]:
df = df.dropna(axis=0)

authors = df['first_author']
papers = defaultdict(list)
authors_id2names = defaultdict(list)

def get_authors_papers(authors):
  for author in authors:
    print(author)
    url = 'https://api.semanticscholar.org/graph/v1/author/search?query=' + str(author) + '&fields=name,papers'
    response = requests.get(url)
    if response.status_code == 200:
        response_data = response.json()
        if 'data' in response_data and response_data['data']:
          first_author_data = response_data['data'][0]
          author_id = first_author_data.get('authorId', None)
          print(author_id)
          papers[author_id].append(first_author_data['papers'])
          authors_id2names[author_id] = author
        else:
          print("No 'data' key in the response.")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")


In [34]:
get_authors_papers(authors)


J. Blagih
4194692
Cen Zhang
2146064158
Juan Liu
2108516226
A. D. Agupitan
1420436824
M. Mijit
6164370
Qi Wu
2143598845
J. Jung
2219590447
Yonghua Shi
9140766
K. R. Patel
121141801
N. Sobhani
2087485610


In [9]:
print(authors)

0          J. Blagih
1          Cen Zhang
4           Juan Liu
6     A. D. Agupitan
8           M. Mijit
9              Qi Wu
12           J. Jung
13       Yonghua Shi
15       K. R. Patel
17        N. Sobhani
Name: first_author, dtype: object


In [35]:
print(papers.keys())

dict_keys(['4194692', '2146064158', '2108516226', '1420436824', '6164370', '2143598845', '2219590447', '9140766', '121141801', '2087485610'])


In [36]:
def get_abstracts_by_paper_id(paper_id):
  url = 'https://api.semanticscholar.org/graph/v1/paper/' + str(paper_id) + '?fields=abstract'
  response = requests.get(url)
  if response.status_code == 200:
      j = response.json()

      # Check if 'data' key exists in the response
      if 'abstract' in j:
          return j['abstract']
      else:
          print("No 'data' key in the response.")
  else:
      print(f"Failed to fetch data. Status code: {response.status_code}")

In [37]:
keys = papers.keys()
abstracts = []
classes = []

for key in keys:
  for papers_with_key in papers[key]:
    for paper in papers_with_key:
      abstract = get_abstracts_by_paper_id(paper['paperId'])
      if abstract is not None:
        abstracts.append(abstract)
        classes.append(key)

In [13]:
print(keys)

dict_keys(['4194692', '2146064158', '2108516226', '1420436824', '6164370', '2143598845', '2219590447', '9140766', '121141801', '2087485610'])


In [14]:
from collections import Counter

def filter_classes_and_metrics(y, X):
    # Count the frequency of each class in y
    class_counts = Counter(y)

    # Filter out classes that appear less than twice and their corresponding X entries
    filtered_y = []
    filtered_X = []
    for class_val, metrics in zip(y, X):
        if class_counts[class_val] > 1:
            filtered_y.append(class_val)
            filtered_X.append(metrics)

    return filtered_y, filtered_X


In [15]:
filtered_classes, filtered_abstracts = filter_classes_and_metrics(classes, abstracts)

In [22]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(filtered_abstracts)
y = filtered_classes




In [23]:
def assign_class_labels(author_ids):
    # Create a mapping of author ID to a unique class integer
    class_mapping = {author_id: idx for idx, author_id in enumerate(sorted(set(author_ids)))}

    # Assign class labels based on the mapping
    class_labels = [class_mapping[author_id] for author_id in author_ids]

    return class_labels, class_mapping

class_labels, class_mapping = assign_class_labels(filtered_classes)
X_train, X_test, y_train, y_test = train_test_split(X, class_labels, test_size=0.2, random_state=42, stratify=y)


In [24]:
default_config = {
    'max_depth'       : 7,
    'learning_rate'   : 0.01,
    'n_estimators'    : 1000,
    'subsample'       : 0.8,
    'min_child_weight': 6,
    'random_state'    : 666
}

In [25]:
model = XGBClassifier(**default_config)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [28]:
predictions = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.65


In [38]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.linear_model import LogisticRegression

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

inputs = tokenizer(filtered_abstracts, padding=True, truncation=True, return_tensors="pt", max_length=512)

with torch.no_grad():
    outputs = model(**inputs)
    encoded_layers = outputs.last_hidden_state

class_labels, class_mapping = assign_class_labels(filtered_classes)

X_bert = torch.mean(encoded_layers, dim=1)
X_bert = X_bert.numpy()
y_bert = filtered_classes

X_train, X_test, y_train, y_test = train_test_split(X_bert, class_labels, test_size=0.2, random_state=42, stratify=y_bert)

classifier = XGBClassifier(**default_config)
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [39]:
predictions = classifier.predict(X_test)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.65


In [19]:
!pip3 install catboost
from catboost import CatBoostClassifier

default_config = {
            'iterations'   : 1000,
            'learning_rate': 0.05,
            'depth'        : 10,
            'loss_function': 'Logloss',
            'eval_metric'  : 'Logloss',
            'random_seed'  : 666,
        }

model_catboost = CatBoostClassifier(**default_config)
model_catboost.fit(X_train, y_train)

prediction_catboost = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_catboost = accuracy_score(y_test, prediction_catboost)

print(prediction_catboost)
print(accuracy_catboost)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 98.7/98.7 MB 109.1 MB/s eta 0:00:01
ERROR: Operation cancelled by user


ModuleNotFoundError: ignored

In [ ]:
!pip3 install lightgbm
from lightgbm import LGBMClassifier
model_lgbm = LGBMClassifier(**default_config)
model_lgbm.fit(X_train, y_train)
print(predictions)
print(accuracy)